# Convolutional neural network

수학적으로 데이터의 지역적 구조를 효과적으로 활용할 수 있는 도구이다.

자연스러운 공분산 구조로 효과적인 학습이 가능하다.

현재 거의 모든 이미지 처리 과정에서 합성곱 신경망을 사용중이다.

그래프 합성곱 아키텍쳐를 사용하면 분자데이터에도 합성곱 네트워크를 사용할 수 있으며, 유전체학, 문자열 처리, 언어번역에서도 성공을 거두고 있다.


## 합성곱 아키텍쳐 소개

합성곱에는 여러 기본요소가 있으며, 기본 요소에는 합성곱 계층, 풀링 계층이 있다.

관련된 용어로는 지역 수용 영역 크기, 스트라이드 크기, 필터 수 등이 있다.

### 지역 수용 영역 (local receptive field)

신경 과학에서 비롯되었으며 각 지역 수용 영역은 이미지의 픽셀 조각에 해당하고 별도의 뉴런이 처리한다.

뉴런은 완전연결 계층과 상당히 유사하다. 비선형 변형을 지역 수용 이미지 조각으로부터 입력된 데이터에 적용한다.

합성곱 뉴런 계층을 결합하면 합성곱 계층이 되며, 이는 공간 지역을 다른 공간 지역으로 변형한다.

지역 수용 영역은 이미지 데이터에만 한정되지 않는다. 합성곱 계층의 출력을 다음 합성곱 계층의 입력이 되는 합성곱 아키텍처에서 처리된 피처데이터 조각은 지역 수용 영역이 될 수 있다.

### 합성곱 커널

합성곱 커널은 가중치 행렬이며 완전연결 계층의 가중치와 관련있다.

합성곱 네트워크의 핵심 아이디어는 '똑같은 비선형 변환이 이미지의 모든 지역수용 영역에 적용된다는 것'이다.

각 지역 수용 영역마다 비선형 함수가 적용되어 이미지 조각이 하나의 수로 반환되는 것을 볼 수 있다. (이를 필터링이라할 수 있다.)

일반적으로 수용 영역의 픽셀 수로 지역 수용 영역의 크기를 정한다. 주로 5 * 5 or 7 * 7을 사용한다.

지역수용 영역이 겹치지 않으려면 합성곱 커널의 스트라이드 크기를 변경하여 간격을 조정한다.

스트라이드 크기를 조절하면 수용영역이 입력 위에서 어떻게 이동하는지 알 수 있다.

정의한 합성곱 커널은 수로 구성된 격자를 다른 수의 격자로 변환한다.

두개 이상의 격자를 출력하려면 합성 커널을 추가하면 된다.

합성곱 커널이 모여 최종적으로 합성곱 계층을 이룬다.
 
다차원 입력에서도 마찬가지로 행렬을 생각하면 좋다. RGB이미지를 예를들면 이늘 3개의 색상채널을 가지므로 랭크3 텐서이다. 5 * 5 픽셀의 지역수용 영역에 3가지 생상 채널이 추가되면 5 * 5 * 3 크기의 지역 수용 영역이 된다.

지역수용 영역이 고차원 텐서라면 스트라이드 크기도 차원에 따른 다차원 스트라이드여야 한다.

합성곱 커널이 너무 많아지면 학습가능한 가중치가 너무 많아져 네트워크가 느려질 수 있다.

### 풀링 계층

계산비용을 줄이는 방법으로 풀링을 사용하며 잘 알려진 고정 비선형 변환으로 최대풀링(Max pooling)이 있다.

각 지역 수용 조각에서 최대 활성입력을 선택하여 출력하는 형태이다.

2 * 2 필터와 스트라이드 2를 사용하여 4 * 4 단일 깊이 조각을 2 * 2로 줄일 수 있다. (이때 필터안에서 가장 큰 값을 출력 값으로 선택)

풀링은 하드웨어가 개선되면서 손실성의 문제로 사용하지 않게 되는 추세지만 그 기능을 숙지할 필요가 있다.

## 합성곱(콘볼루션) 네트워크 구축

합성곱 계층과 풀링 계층으로 이루어진 간단한 합성곱 아키텍처를 구축한다.

아키텍처 설계의 핵심은 '정교한 형태의 레고 쌓기'이다.

구축할 아키텍처의 형태는 입력-합성곱층1-풀링층1-합성곱층2-풀링층2-은닉층4-출력 형태이며 은닉층과 출력은 풀리 커넥티드로 구성한다.

다음은 간단한 합성곱 신경망을 학습하는 코드 샘플을 통해 실습을 진행한다.

코드 샘플은 텐서플로를 이용하여 LeNet-5 합성곱 아키텍처를 MNIST 데이터에 대해 학습시키는 것을 보여준다.

#### 해당 코드 소스는 python 3.6, tensorflow 1.8.0 버전에서 구동된다. 아나콘다 프롬프트에서 해당 가상환경을 생성하여 실습을 진행한다.

In [1]:
## 기본 import 및 파라미터를 설정
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import sys
import time

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/' # mnist 데이터 소스 경로
WORK_DIRECTORY = 'data' # 데이터 소스 저장폴더 경로
IMAGE_SIZE = 28 # 이미지 사이즈. mnist는 28*28이다.
NUM_CHANNELS = 1 # 채널수
PIXEL_DEPTH = 255 # 입력받은 픽셀은 0~255의 범위를 갖는다. 입력 픽셀의 깊이는 0~255로 딥네트워크가 불안해질 수 있다. 
NUM_LABELS = 10 # 라벨 갯수. 0~9까지의 10개이다.
VALIDATION_SIZE = 5000  # 검증 데이터셋 크기
SEED = 66478  # 랜덤 시드 세팅.
BATCH_SIZE = 64 # 미니배치 사이즈
NUM_EPOCHS = 10 
EVAL_BATCH_SIZE = 64 # 평가 미니배치 사이즈
EVAL_FREQUENCY = 100 # 평가 스텝 갯수.

### MNIST 적재

웹사이트 https://yann.lecun.com/exdb/mnist/ 에서 찾을 수 있다.

다운로드 스크립트로 웹사이트에서 원본 파일을 내려받을 수 있다. 스크립트는 캐시를 사용하여 여러번 download()를 호출해도 낭비가 아니다.

일반적으로 ML 데이터 집합을 클라우드에 저장하고 학습 알고리즘의 입력으로 처리하기 전에 사용자 코드를 가져온다.

분석을 위해 커다란 데이터를 호스팅하려면 클라우드를 이용해서 필요할때 마다 로컬 장비에 내려받는 것이 좋다.

In [2]:
## mnist 데이터 집합 다운로드 함수
def download(filename):
    """데이터가 존제하지 않으면, 얀의 웹사이트에서 다운로드 수행."""
    if not os.path.exists(WORK_DIRECTORY): # 저장폴더가 없을 경우
        os.makedirs(WORK_DIRECTORY) # 저장 폴더 생성
    filepath = os.path.join(WORK_DIRECTORY, filename) # 데이터셋의 경로를 path로 설정
    
    if not os.path.exists(filename): # 저장된 데이터셋이 없을 경우
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath) # 소스 url에서 데이터를 다운로드
        size = os.stat(filepath).st_size
        print('Successfully downloaded', filename, size, 'bytes.')
        
    return filepath

### 데이터셋 추출

mnist 데이터 집합은 픽셀 값을 인코딩한 원본 문자열의 바이트로 저장되며, 이 데이터를 넘파이 배열로 변환한다.

In [3]:
def extract_data(filename, num_images):
    """이미지를 4차원 텐서[이미지 인덱스, y, x, 채널]로 추출한다.
    
    값의 범위를 [0, 255]에서 -0.5, 0.5로 줄인다.
    """
    
    print('Extracting', filename)
    with gzip.open(filename) as bytestream: # gzip 파일을 압축 해제한다.
        bytestream.read(16)
        buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
        data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32) # np.frombuffer 함수로 원본 바이트 버퍼를 숫자배열로 변환.
        data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH # 안정성을 위해 평균 0에 단위 범위를 (-0.5, 0.5)로 조절한다.
        data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
        return data

다운로드한 데이터 집합에서 레이블 배열 추출

In [4]:
def extract_labels(filename, num_images):
    """int64형 레이블 ID벡터로 레이블을 추출"""
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(8) # 8비트 헤더가 있고 나머지 데이터는 레이블이다.
        buf = bytestream.read(1 * num_images)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64) # 레이블을 저장한다.
    return labels

이전 위에서 정의한 함수로 mnist학습과 검증 데이터 집합을 다운로드하고 처리

In [5]:
# 데이터를 가져옴
train_data_filename = download('train-images-idx3-ubyte.gz') 
train_labels_filename = download('train-labels-idx1-ubyte.gz')
test_data_filename = download('t10k-images-idx3-ubyte.gz')
test_labels_filename = download('t10k-labels-idx1-ubyte.gz')

# 넘파이 배열로 추출
train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


mnist 데이터 집합은 하이퍼 파라미터 튜닝을 위한 검증데이터 집합이 명확하지 않으므로,

학습데이터 집합의 최종 5천개 데이터를 검증데이터로 수동지정한다.

In [6]:
# 검증 집합 생성
validation_data = train_data[:VALIDATION_SIZE, ...] # 리스트 슬라이싱 리스트[처음인덱스:마지막인덱스] 마지막인덱스는 마지막요소인덱스+1
validation_labels = train_labels[:VALIDATION_SIZE] # [:VALIDATION_SIZE]이면 앞에서 부터 시작
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:] # # [VALIDATION_SIZE:]이면 뒤에서 부터 시작 

## 에폭 크기 및 트레이닝 사이즈 정의
num_epochs = NUM_EPOCHS
train_size = train_labels.shape[0]

### 텐서플로 합성곱 기본 요소

콘볼루션 네트워크를 구축할때 필요한 기본 요소를 설계한다.

기본 네트워크 아키텍처는 tf.nn.conv2d, tf.nn.max_pool, 비선형성, 완전연결 계층이 서로 섞여있다.

정규화를 위해 학습 단계에서만 드롭아웃 계층을 마지막 완전연결 계층 이후에 적용 시킨다.

각 레이어를 생성하고 레이어를 서로 연결시켜 콘볼루션 네트워크를 구성한다.

In [7]:
## 완전연결 계층을 위한 학습 가능 가중치 정의
def model(data, train=False):
    ## 2차원 합성곱 레이어
    # 첫번째 합성곱 레이어
    # conv2d 메서드는 텐서플로 내장함수로 합성곱 계층을 정의한다.
    conv = tf.nn.conv2d(data, # input. 텐서 형상(batch, height, width, channels)를 가정한다. batch는 미니배치 수이다.
                        conv1_weights, # filters
                        strides = [1,1,1,1], # 필터 스트라이드. 형상은 데이터 레이아웃과 동일한 [image_index, y, x, 깊이]이다.
                        padding = 'SAME', # 입력 텐서에 여백을 넣는다. padding="SAME"이면 입력과 출력이 동일하도록 여백을 넣으며, 
                                          # padding="VALID"일 경우 추가 여백은 넣지 않는다.
                        use_cudnn_on_gpu=True) #, # 생략가능
                        # data_format=None, # 생략가능
                        # name=None) # 생략가능
    
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases)) # 활성화함수 정의.
        
    ## 최대 풀링 레이어
    pool = tf.nn.max_pool(relu, # value. tf.nn.conv2d의 입력과 동일한 형상.
                          ksize = [1,2,2,1], # 풀링 창의 크기이며 크기 4의 리스트이다. 데이터 레이아웃과 같다.
                          strides = [1,2,2,1], # 풀링 창은 2이며 스트라이드는 2이다.
                          padding = 'SAME',
                          data_format='NHWC', # 생략가능
                          name=None) # 생략가능
    
    # 두번째 합성곱 레이어
    conv = tf.nn.conv2d(pool,
                       conv2_weights,
                       strides=[1,1,1,1],
                       padding='SAME',
                       use_cudnn_on_gpu=True)
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    
    pool = tf.nn.max_pool(relu,
                          ksize=[1,2,2,1],
                          strides=[1,2,2,1],
                          padding='SAME')
    
    # 직육면체 피처 맵을 2d 행렬로 변환하고 완전연결 계층으로 전달
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool,
                        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    
    # 완전연결 계층. + 연산은 자동으로 편향에 브로드캐스트 된다.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    
    # 학습중 50%의 드롭아웃을 추가. 드롭아웃은 평가시점에서는 조절이 필요 없도록 활성화 정도를 조절한다.
    if train: # train이 true값일 때
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
        
    return tf.matmul(hidden, fc2_weights) + fc2_biases

### 합성곱 네트워크의 학습 가능 가중치 정의

정의한 아키텍처에서 두개의 풀링계층이 두개의 합성곱 계층 사이에 교차하며 배치되고, 마지막으로 두개의 완전연결 계층이 있다.

풀링은 학습가능한 가중치가 없고, 합성곱 및 완전연결 계층에서만 가중치를 지정한다.(tf.nn.conv2d에서 filter 인수에 대응하는 학습가능 텐서)

합성곱 편향(bias)을 각 출력 채널에 추가한다.

In [8]:
## 합성곱 계층을 위한 학습 가능 가중치 정의
conv1_weights = tf.Variable(
    tf.truncated_normal([5,5,NUM_CHANNELS,32], # 5*5 filter이며 깊이는 32이다. 4-텐서
                       stddev = 0.1,
                       seed=SEED, dtype=tf.float32))

conv1_biases = tf.Variable(tf.zeros([32], dtype=tf.float32)) # 1-텐서

conv2_weights = tf.Variable(
    tf.truncated_normal([5,5,32,64], # 5*5 filter이며, conv1의 결과에 맞춰 채널을 32로 두고 깊이 64를 갖도록 세팅한다.
                       stddev = 0.1,
                       seed=SEED, dtype=tf.float32))

conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=tf.float32))

첫번째 완전연결 계층은 콘볼루션 계층의 출력을 512 크기 벡터로 변환한다.

제일 처음 입력 이미지는 IMAGE_SIZE = 28크기로 시작하고 두개의 풀링 계층(각각 입력을 1/2로 줄인다)을 지나면 이미지 크기는 IMAGE // 4 가 된다.

완전연결 가중치의 형상은 이에 맞춰 생성한다.

두번째 완전연결 계층은 출력으로 10개의 label을 주어야 하므로 가중치형상은 (512,10)이며 편향 형상은 (10)이 된다.

In [9]:
## 완전연결 계층을 위한 학습 가능한 가중치 정의
fc1_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 512], # 두번의 풀링 계층을 지나 이미지 크기가 줄어듬.
                                              stddev=0.1,
                                              seed=SEED,
                                              dtype=tf.float32)) 

fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))

fc2_weights = tf.Variable(tf.truncated_normal([512, NUM_LABELS], # 입력 512, 깊이는 라벨 갯수
                                              stddev=0.1,
                                              seed=SEED,
                                              dtype=tf.float32)) 

fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS], dtype=tf.float32))

### 플레이스홀더 정의

플레이스홀더는 학습할 이미지와 그 레이블을 입력받는다.

특수 네트워크에서 커다란 배치 입력을 평가할 수 있도록 별도의 플레이스홀더를 정의한다.

In [10]:
## 아키텍처 플레이스홀더 정의
train_data_node = tf.placeholder(tf.float32,
                                shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))
eval_data = tf.placeholder(tf.float32,
                          shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))

### logits와 loss 함수 정의

학습에 필요한 기타 함수들의 정의를 진행한다.

In [11]:
## logits, corss-entorpy loss 정의
logits = model(train_data_node, True)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=train_labels_node, logits=logits))

## fully connected parameters를 위한 L2 정규화 정의
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases)
               + tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))

## loss에 regularization term을 더한다.
loss += 5e-4 * regularizers

W0716 17:17:36.442596 13704 deprecation.py:506] From <ipython-input-7-eaeae05b9ee2>:48: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### optimizer 정의

배치마다 increment를 수행하여 variable을 셋업하고 learning rate를 조정한다.

In [12]:
## 배치사이즈
batch = tf.Variable(0, dtype=tf.float32)

## 에폭마다 Decay를 한번 수행, 0.01부터 시작하여 exponential schedule을 사용한다.
learning_rate = tf.train.exponential_decay( 0.01, # base learning rate
                                          batch * BATCH_SIZE, # dataset 안에서 현재 index
                                          train_size, # decay 스텝
                                          0.95, # decay rate
                                          staircase = True)

## optimization을 위한 simple momentum을 사용
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

## 현재 학습 미니배치에서 예측값
train_prediction = tf.nn.softmax(logits)

## 신경망 학습

처리할 데이터, 입력, 가중치, 모델을 모두 정의하면 데이터셋을 신경망에 입력하여 학습을 시작한다.

### LeNet-5 아키텍처 학습

5 계층으로 구성된 신경망 학습을 수행

In [13]:
# 학습 수행을 위해 지역 세션을 생성
start_time = time.time()
with tf.Session() as sess:
    # 모든 initializers를 실행해 학습가능한 variable을 준비.
    tf.global_variables_initializer().run()
    
    # 학습단계 순환. 미니 배치 수행
    for step in xrange(int(num_epochs * train_size) // BATCH_SIZE): # 수행할 전체 에폭*학습크기를 배치사이즈로 나눔. 에폭마다 더 나은 무작위 추출을 사용할 수도 있다.
        offset = (step*BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        
        # 피드딕트로 배치 데이터(넘파이 배열)를 그래프에 전달할 노드에 매핑한다.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}
        
        # 가중치 업데이트를 위해 옵티마이저 수행
        sess.run(optimizer, feed_dict=feed_dict)
        

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Conv2D (defined at <ipython-input-7-eaeae05b9ee2>:11) ]]

Errors may have originated from an input operation.
Input Source operations connected to node Conv2D:
 Placeholder (defined at <ipython-input-10-628dd8bfede7>:3)	
 Variable/read (defined at <ipython-input-8-aaa3ed07811e>:5)

Original stack trace for 'Conv2D':
  File "D:\_setup\anaconda3\envs\py3.7\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\_setup\anaconda3\envs\py3.7\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\_setup\anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\_setup\anaconda3\envs\py3.7\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\_setup\

### 학습된 모델 평가

학습 모델의 정확도를 평가하기 휘해서는 에러 지표를 정의하는 것이 좋다.

In [15]:
## 학습된 아키텍처 에러 평가
def error_rate(predictions, labels):
    """조밀한 예측과 희박한 레이블의 오차율을 반환한다."""
    return 100.0 - (100.0 * numpy.sum(numpy.argmax(predictions, 1) == labels) / predictions.shape[0])
# 에러를 평가할 수 있는 함수. 

임의의 배치 데이터 집합에 대한 예측을 평가하는 별도의 함수는 네트워크는 고정된 배치 크기의 입력만 다룰 수 있으므로 

softmax 함수를 사용하여 편하게 확인할 수 있다.

In [16]:
# 평가 예측 노드
eval_prediction = tf.nn.softmax(model(eval_data))

def eval_in_batches(data, sess):
    """미니배치로 수행하여 데이터 집합에 대한 예측을 얻는다."""
    size = data.shape[0] # 데이터 크기를 size로 받아온다
    if size < EVAL_BATCH_SIZE: # 사이즈가 미니배치 사이즈보다 작을경우 벨류 에러르 발생한다.
        raise ValueError("batch sizer for evals larger than dataset: %d" % size)
        
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), # 예측값이 저장될 predictions 를 넘파이 어레이로 초기화.
                               dtype=numpy.float32)
    
    for begin in xrange(0, size, EVAL_BATCH_SIZE): # 0부터 배치사이즈간격으로 수행
        end = begin + EVAL_BATCH_SIZE # end는 배치의 끝
        if end <= size: # end가 사이즈보다 커지면 else를 수행.
            predictions[begin:end,:] = sess.run(eval_prediction, 
                                               feed_dict={eval_data: data[begin:end, ...]})
            
        else: # 남은 사이즈가 배치 배치 사이즈보다 작을경우엔 남은 부분만 예측을 수행한다.
            batch_predictions = sess.run(eval_prediction,
                                        feed_dict = {eval_data: data[-EVAL_BATCH_SIZE:, ...]})
            predictions[begin:,:] = batch_predictions[begin - size:, :]
    
    return predictions

### 주기적 모델 평가

주기적으로 검증 집합에 대한 모델 정확도를 평가하려면 학습 내부의 for 순환문에서 약간의 조작을 수행한다.

실험 정확도 점수를 매기고 학습을 마친다.

In [21]:
## 조작이 포함된 네트워크를 학습하는 완전한 코드
start_time = time.time()
with tf.Session() as sess:
    # 모든 initializers를 실행해 학습가능한 variable을 준비.
    tf.global_variables_initializer().run()
    
    # 학습단계 순환. 미니 배치 수행
    for step in xrange(int(num_epochs * train_size) // BATCH_SIZE): # 수행할 전체 에폭*학습크기를 배치사이즈로 나눔. 
        offset = (step*BATCH_SIZE) % (train_size - BATCH_SIZE)      # 에폭마다 더 나은 무작위 추출을 사용할 수도 있다.
        batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        
        # 피드딕트로 배치 데이터(넘파이 배열)를 그래프에 전달할 노드에 매핑한다.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}
        
        # 가중치 업데이트를 위해 옵티마이저 수행
        sess.run(optimizer, feed_dict=feed_dict)
        
        ## 추가되는 함수
        if step % EVAL_FREQUENCY == 0:
            # 추가 노드 정보를 가져온다.
            l, lr, predictions = sess.run([loss, learning_rate, train_prediction], feed_dict = feed_dict)
            
            # 경과 시간 체크
            elapsed_time = time.time() - start_time
            start_time = time.time()
            print('Step %d (epoch %.2f), %.1f ms' % (step,float(step) * BATCH_SIZE / train_size, 1000 * elapsed_time / EVAL_FREQUENCY))
            print('Minibatch loss: %.3f, learning rate: %.6f' % (l,lr)) # 미니배치의 로스 출력
            print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels)) # 에러레이트 함수로 미니배치의 에러 출력
            print('Validation error: %.1f%%' % error_rate(eval_in_batches(validation_data, sess), validation_labels))
            sys.stdout.flush()
            
        # 마지막 결과 출력
        test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
        print('Test label: %d, Accuracy: %.1f%%' % (test_labels[step], 100-test_error))
        
    


Step 0 (epoch 0.00), 2.5 ms
Minibatch loss: 8.334, learning rate: 0.010000
Minibatch error: 85.9%
Validation error: 84.5%
Test label: 7, Accuracy: 16.0%
Test label: 2, Accuracy: 24.8%
Test label: 1, Accuracy: 21.9%
Test label: 0, Accuracy: 30.7%
Test label: 4, Accuracy: 34.1%
Test label: 1, Accuracy: 34.1%
Test label: 4, Accuracy: 31.7%
Test label: 9, Accuracy: 34.4%
Test label: 5, Accuracy: 43.1%
Test label: 9, Accuracy: 42.2%
Test label: 0, Accuracy: 46.1%
Test label: 6, Accuracy: 49.5%
Test label: 9, Accuracy: 52.4%
Test label: 0, Accuracy: 56.5%
Test label: 1, Accuracy: 59.7%
Test label: 5, Accuracy: 62.4%
Test label: 9, Accuracy: 64.8%
Test label: 7, Accuracy: 67.3%
Test label: 3, Accuracy: 69.5%
Test label: 4, Accuracy: 72.3%
Test label: 9, Accuracy: 74.4%
Test label: 6, Accuracy: 75.4%
Test label: 6, Accuracy: 75.6%
Test label: 5, Accuracy: 75.7%
Test label: 4, Accuracy: 76.1%
Test label: 0, Accuracy: 76.7%
Test label: 7, Accuracy: 77.0%
Test label: 4, Accuracy: 78.8%
Test label

Test label: 5, Accuracy: 95.7%
Test label: 7, Accuracy: 95.9%
Test label: 7, Accuracy: 96.0%
Test label: 2, Accuracy: 96.2%
Test label: 8, Accuracy: 96.3%
Test label: 2, Accuracy: 96.5%
Test label: 6, Accuracy: 96.6%
Test label: 8, Accuracy: 96.6%
Test label: 5, Accuracy: 96.7%
Test label: 7, Accuracy: 96.6%
Test label: 7, Accuracy: 96.5%
Test label: 9, Accuracy: 96.4%
Test label: 1, Accuracy: 96.2%
Test label: 8, Accuracy: 96.3%
Test label: 1, Accuracy: 96.5%
Test label: 8, Accuracy: 96.5%
Test label: 0, Accuracy: 96.6%
Test label: 3, Accuracy: 96.7%
Test label: 0, Accuracy: 96.7%
Test label: 1, Accuracy: 96.6%
Test label: 9, Accuracy: 96.5%
Test label: 9, Accuracy: 96.5%
Test label: 4, Accuracy: 96.3%
Test label: 1, Accuracy: 95.9%
Test label: 8, Accuracy: 95.6%
Test label: 2, Accuracy: 95.3%
Test label: 1, Accuracy: 95.1%
Test label: 2, Accuracy: 95.0%
Test label: 9, Accuracy: 95.0%
Test label: 7, Accuracy: 95.2%
Test label: 5, Accuracy: 95.6%
Test label: 9, Accuracy: 95.8%
Test lab

Test label: 1, Accuracy: 97.7%
Test label: 3, Accuracy: 97.6%
Test label: 6, Accuracy: 97.6%
Test label: 5, Accuracy: 97.6%
Test label: 7, Accuracy: 97.7%
Test label: 4, Accuracy: 97.7%
Test label: 2, Accuracy: 97.7%
Test label: 2, Accuracy: 97.5%
Test label: 6, Accuracy: 97.5%
Test label: 3, Accuracy: 97.5%
Test label: 2, Accuracy: 97.5%
Test label: 6, Accuracy: 97.5%
Test label: 5, Accuracy: 97.4%
Test label: 4, Accuracy: 97.4%
Test label: 8, Accuracy: 97.5%
Test label: 9, Accuracy: 97.5%
Test label: 7, Accuracy: 97.5%
Test label: 1, Accuracy: 97.5%
Test label: 3, Accuracy: 97.5%
Test label: 0, Accuracy: 97.5%
Test label: 3, Accuracy: 97.5%
Test label: 8, Accuracy: 97.5%
Test label: 3, Accuracy: 97.5%
Test label: 1, Accuracy: 97.5%
Test label: 9, Accuracy: 97.5%
Test label: 3, Accuracy: 97.5%
Test label: 4, Accuracy: 97.5%
Test label: 4, Accuracy: 97.6%
Test label: 6, Accuracy: 97.7%
Test label: 4, Accuracy: 97.7%
Test label: 2, Accuracy: 97.7%
Test label: 1, Accuracy: 97.5%
Test lab

Test label: 0, Accuracy: 98.2%
Test label: 2, Accuracy: 98.2%
Test label: 4, Accuracy: 98.2%
Test label: 5, Accuracy: 98.2%
Test label: 1, Accuracy: 98.2%
Test label: 1, Accuracy: 98.2%
Test label: 6, Accuracy: 98.2%
Test label: 4, Accuracy: 98.2%
Test label: 7, Accuracy: 98.2%
Test label: 1, Accuracy: 98.2%
Test label: 9, Accuracy: 98.2%
Test label: 4, Accuracy: 98.2%
Test label: 2, Accuracy: 98.2%
Test label: 4, Accuracy: 98.3%
Test label: 1, Accuracy: 98.3%
Test label: 5, Accuracy: 98.3%
Test label: 5, Accuracy: 98.3%
Test label: 3, Accuracy: 98.3%
Test label: 8, Accuracy: 98.3%
Test label: 3, Accuracy: 98.3%
Test label: 1, Accuracy: 98.3%
Test label: 4, Accuracy: 98.3%
Test label: 5, Accuracy: 98.2%
Test label: 6, Accuracy: 98.2%
Test label: 8, Accuracy: 98.2%
Test label: 9, Accuracy: 98.2%
Test label: 4, Accuracy: 98.2%
Test label: 1, Accuracy: 98.2%
Test label: 5, Accuracy: 98.2%
Test label: 3, Accuracy: 98.1%
Test label: 8, Accuracy: 98.1%
Test label: 0, Accuracy: 98.1%
Test lab

Test label: 2, Accuracy: 98.5%
Test label: 6, Accuracy: 98.5%
Test label: 8, Accuracy: 98.5%
Test label: 1, Accuracy: 98.5%
Test label: 3, Accuracy: 98.5%
Test label: 7, Accuracy: 98.5%
Test label: 5, Accuracy: 98.5%
Test label: 4, Accuracy: 98.5%
Test label: 4, Accuracy: 98.4%
Test label: 1, Accuracy: 98.4%
Test label: 8, Accuracy: 98.4%
Test label: 1, Accuracy: 98.4%
Test label: 3, Accuracy: 98.4%
Test label: 8, Accuracy: 98.4%
Test label: 1, Accuracy: 98.3%
Test label: 2, Accuracy: 98.4%
Test label: 5, Accuracy: 98.4%
Test label: 8, Accuracy: 98.4%
Test label: 0, Accuracy: 98.3%
Test label: 6, Accuracy: 98.3%
Test label: 2, Accuracy: 98.3%
Test label: 1, Accuracy: 98.3%
Test label: 1, Accuracy: 98.3%
Test label: 7, Accuracy: 98.3%
Test label: 1, Accuracy: 98.3%
Test label: 5, Accuracy: 98.4%
Test label: 3, Accuracy: 98.3%
Test label: 4, Accuracy: 98.4%
Test label: 6, Accuracy: 98.3%
Test label: 9, Accuracy: 98.3%
Test label: 5, Accuracy: 98.4%
Test label: 0, Accuracy: 98.4%
Test lab

...

Test label: 7, Accuracy: 99.2%
Test label: 4, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 5, Accuracy: 99.2%
Test label: 4, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 3, Accuracy: 99.2%
Test label: 5, Accuracy: 99.2%
Test label: 3, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Step 7400 (epoch 8.61), 279.2 ms
Minibatch loss: 1.700, learning rate: 0.006634
Minibatch error: 0.0%
Validation error: 0.7%
Test label: 2, Accuracy: 99.2%
Test label: 3, Accuracy: 99.2%
Test label: 4, Accuracy: 99.3%
Test label: 5, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 2, Accuracy: 99.2%
Test label: 3, Accuracy: 99.2%
Test label: 4, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 0, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 2, Accuracy: 99.2%
Test label: 4, Accuracy: 99.2%
Test label: 5, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 8, Accuracy: 99.2%
Test la

Test label: 5, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 6, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 4, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 0, Accuracy: 99.2%
Test label: 5, Accuracy: 99.2%
Test label: 5, Accuracy: 99.2%
Test label: 3, Accuracy: 99.2%
Test lab

Test label: 1, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 8, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 8, Accuracy: 99.2%
Test label: 2, Accuracy: 99.2%
Test label: 8, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 6, Accuracy: 99.2%
Test label: 8, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 6, Accuracy: 99.2%
Test label: 2, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test lab

Test label: 8, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 3, Accuracy: 99.2%
Test label: 0, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 2, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test lab

Test label: 8, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 5, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 0, Accuracy: 99.2%
Test label: 5, Accuracy: 99.3%
Test label: 4, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 1, Accuracy: 99.3%
Test label: 3, Accuracy: 99.2%
Test label: 1, Accuracy: 99.2%
Test label: 7, Accuracy: 99.2%
Test label: 9, Accuracy: 99.2%
Test label: 6, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 0, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 7, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test label: 4, Accuracy: 99.3%
Test label: 3, Accuracy: 99.3%
Test label: 9, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 2, Accuracy: 99.3%
Test label: 1, Accuracy: 99.3%
Test label: 8, Accuracy: 99.3%
Test label: 6, Accuracy: 99.3%
Test lab